# **Model Training:**

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import label_binarize
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, roc_curve, auc, precision_recall_fscore_support, confusion_matrix
import os
import gc
import shutil
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.stats import mode
import seaborn as sns
import joblib
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input as preprocess_input_vgg
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input as preprocess_input_inception
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as preprocess_input_resnet
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input as preprocess_input_densenet
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
base_dir = "/content/drive/MyDrive/Data 298B Project Data/Rice Image Datasets - with Location and Time/Rice Leaf Diseases - Taiwan Filtered"

#df = pd.read_csv(f"{base_dir}/main_df_rice.csv")
df = pd.read_csv(f"{base_dir}/image_metadata_taiwan_filtered_location.csv")
train_df = pd.read_csv(f"{base_dir}/train_df_with_augmentation_rice.csv")
val_df = pd.read_csv(f"{base_dir}/val_df_rice.csv")
test_df = pd.read_csv(f"{base_dir}/test_df_rice.csv")

train_image_dir = f'{base_dir}/Train_Dataset_with_Augmentation_Rice'
val_image_dir = f'{base_dir}/Validation_Dataset_Rice'
test_image_dir = f'{base_dir}/Test_Dataset_Rice'

batch_size = 4

In [ ]:
# Label Encoding the Class Variable
label_encoder = LabelEncoder()
labels = df['Class'].unique()
label_encoder.fit(labels)

# Saving the label encodings to a file

joblib.dump(label_encoder, f"{base_dir}/label_encoder.joblib")

In [ ]:
# Loading the label encodings from the previously made joblib file

label_encoder = joblib.load(f'{base_dir}/label_encoder.joblib')

# **VGG19 Model:**

In [ ]:
# Function to preprocess the images to meet the requirements of the pertinent pre-trained model
def preprocess_image(file_path, label):
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize_with_pad(img, 224, 224, antialias=True) # Resizing to 224x224 for VGG19
  img = preprocess_input_vgg(img)  # Applying the preprocess function for VGG19
  return img, label

# Function to prepare the train, validation and test datasets to be compatible with TensorFlow
def prepare_dataset(df, image_dir, preprocessing_function, batch_size=32, cache=True, shuffle=False):
    image_paths = df['Id'].apply(lambda x: f"{image_dir}/{x}").values
    labels = label_encoder.transform(df['Class'].values)
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
    dataset = dataset.map(preprocessing_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    if cache:
        dataset = dataset.cache()
    return dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# Preparing the train, validation and test datasets, with shuffling for the train dataset
train_dataset = prepare_dataset(train_df, train_image_dir, preprocess_image, batch_size=batch_size, cache=True, shuffle=True)
val_dataset = prepare_dataset(val_df, val_image_dir, preprocess_image, batch_size=batch_size, cache=True)
test_dataset = prepare_dataset(test_df, test_image_dir, preprocess_image, batch_size=batch_size, cache=True)

In [ ]:
# Defining the parameter grid with various model combinations
neuron_options = [2048, 4096]
activation_options = ['relu', 'tanh']
dropout_options = [0, 0.3, 0.5]
# Loading the base VGG19 model to determine the total number of layers
temp_model = VGG19(weights='imagenet', include_top=False)
total_layers = len(temp_model.layers)
frozen_layers_options = [total_layers, total_layers - 2, total_layers - 5]  # Experimenting with the number of frozen layers for further fine-tuning

# Initializing a DataFrame to store the results of each model combination
results_df = pd.DataFrame(columns=['Neurons', 'Activation', 'Dropout', 'Frozen Layers', 'Test Accuracy', 'Test Loss'])

# Variables to keep track of the best model's accuracy and details
best_accuracy = 0
best_model_details = {}

for neurons in neuron_options:
    for activation in activation_options:
        for dropout_rate in dropout_options:
            for frozen_layers in frozen_layers_options:

                # Loading the base VGG19 model
                base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

                # Freezing the layers for further fine-tuning
                for layer in base_model.layers[:frozen_layers]:
                    layer.trainable = False

                # Adding custom layers
                x = base_model.output
                x = Flatten()(x)
                x = Dense(neurons, activation=activation)(x)
                x = Dropout(dropout_rate)(x)
                predictions = Dense(len(label_encoder.classes_), activation='softmax')(x)

                # Complete model
                model = Model(inputs=base_model.input, outputs=predictions)
                model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

                # Fitting the model
                history = model.fit(
                    train_dataset,
                    validation_data=val_dataset,
                    epochs=25,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)],
                    verbose = 0
                )

                # Evaluating the model on the test set
                test_loss, test_accuracy = model.evaluate(test_dataset, verbose = 0)
                print(f"Test accuracy: {test_accuracy * 100:.2f}%, Test loss: {test_loss:.4f}")

                # Updating the results DataFrame
                results_df = results_df.append({
                    'Neurons': neurons,
                    'Activation': activation,
                    'Dropout': dropout_rate,
                    'Frozen Layers': frozen_layers,
                    'Test Accuracy': test_accuracy,
                    'Test Loss': test_loss
                }, ignore_index=True)

                # Saving the best model
                if test_accuracy > best_accuracy:
                    best_accuracy = test_accuracy
                    best_model_details = {
                        'Neurons': neurons,
                        'Activation': activation,
                        'Dropout': dropout_rate,
                        'Frozen Layers': frozen_layers,
                        'Test Set Accuracy': f"{test_accuracy * 100:.2f}%"
                    }
                    model.save(f'{base_dir}/Best_VGG19_Visual_Model.h5')

Test accuracy: 89.60%, Test loss: 7.3119


<ipython-input-7-f757cffcc1b1>:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Test accuracy: 91.20%, Test loss: 0.7600


<ipython-input-7-f757cffcc1b1>:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Test accuracy: 8.80%, Test loss: 1.0998


...

<ipython-input-7-f757cffcc1b1>:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


Test accuracy: 84.00%, Test loss: 0.3388


<ipython-input-7-f757cffcc1b1>:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


Test accuracy: 68.00%, Test loss: 0.7355


<ipython-input-7-f757cffcc1b1>:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


In [ ]:
# Displaying all results
print(results_df)

   Neurons Activation Dropout Frozen Layers  Test Accuracy  Test Loss
0     2048       relu       0            22          0.896   7.311944
1     2048       relu       0            20          0.912   0.760017
2     2048       relu       0            17          0.088   1.099824
3     2048       relu     0.3            22          0.880   8.569437
4     2048       relu     0.3            20          0.904   1.601842
5     2048       relu     0.3            17          0.088   1.101093
6     2048       relu     0.5            22          0.888   4.647356
7     2048       relu     0.5            20          0.872   1.841907
8     2048       relu     0.5            17          0.368   1.099905
9     2048       tanh       0            22          0.912   0.214835
10    2048       tanh       0            20          0.872   0.397380
11    2048       tanh       0            17          0.368   1.112538
12    2048       tanh     0.3            22          0.888   0.241474
13    2048       tan

In [ ]:
# Displaying the best model's details
print("Best model details:", best_model_details)

Best model details: {'Neurons': 2048, 'Activation': 'relu', 'Dropout': 0, 'Frozen Layers': 20, 'Test Set Accuracy': '91.20%'}


# **InceptionV3 Model:**

In [4]:
# Function to preprocess the images to meet the requirements of the pertinent pre-trained model
def preprocess_image(file_path, label):
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize_with_pad(img, 299, 299, antialias=True) # Resizing to 299x299 for InceptionV3
  img = preprocess_input_inception(img)  # Applying the preprocess function for InceptionV3
  return img, label

# Label Encoding the Class Variable
label_encoder = LabelEncoder()
labels = df['Class'].unique()
label_encoder.fit(labels)

# Function to prepare the train, validation and test datasets to be compatible with TensorFlow
def prepare_dataset(df, image_dir, preprocessing_function, batch_size=32, cache=True, shuffle=False):
    image_paths = df['Id'].apply(lambda x: f"{image_dir}/{x}").values
    labels = label_encoder.transform(df['Class'].values)
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
    dataset = dataset.map(preprocessing_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    if cache:
        dataset = dataset.cache()
    return dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [5]:
# Preparing the train, validation and test datasets, with shuffling for the train dataset
train_dataset = prepare_dataset(train_df, train_image_dir, preprocess_image, batch_size=batch_size, cache=True, shuffle=True)
val_dataset = prepare_dataset(val_df, val_image_dir, preprocess_image, batch_size=batch_size, cache=True)
test_dataset = prepare_dataset(test_df, test_image_dir, preprocess_image, batch_size=batch_size, cache=True)

In [6]:
# Clearing unnecessary memory usage before training the model
gc.collect()
tf.keras.backend.clear_session()

In [7]:
# Defining the parameter grid with various model combinations
neuron_options = [256, 512]
activation_options = ['relu']
dropout_options = [0, 0.3, 0.5]
# Loading the base InceptionV3 model to determine the total number of layers
temp_model = InceptionV3(weights='imagenet', include_top=False)
total_layers = len(temp_model.layers)
frozen_layers_options = [total_layers, total_layers - 2, total_layers - 5]  # Experimenting with the number of frozen layers for further fine-tuning

# Initializing a DataFrame to store the results of each model combination
results_df = pd.DataFrame(columns=['Neurons', 'Activation', 'Dropout', 'Frozen Layers', 'Test Accuracy', 'Test Loss'])

# Variables to keep track of the best model's accuracy and details
best_accuracy = 0
best_model_details = {}

for neurons in neuron_options:
    for activation in activation_options:
        for dropout_rate in dropout_options:
            for frozen_layers in frozen_layers_options:

                # Loading the base InceptionV3 model
                base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

                # Freezing the layers for further fine-tuning
                for layer in base_model.layers[:frozen_layers]:
                    layer.trainable = False

                # Adding custom layers
                x = base_model.output
                x = Flatten()(x)
                x = Dense(neurons, activation=activation)(x)
                x = Dropout(dropout_rate)(x)
                predictions = Dense(len(label_encoder.classes_), activation='softmax')(x)

                # Complete model
                model = Model(inputs=base_model.input, outputs=predictions)
                model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

                # Fitting the model
                history = model.fit(
                    train_dataset,
                    validation_data=val_dataset,
                    epochs=25,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)],
                    verbose = 0
                )

                # Evaluating the model on the test set
                test_loss, test_accuracy = model.evaluate(test_dataset, verbose = 0)
                print(f"Test accuracy: {test_accuracy * 100:.2f}%, Test loss: {test_loss:.4f}")

                # Updating the results DataFrame
                results_df = results_df.append({
                    'Neurons': neurons,
                    'Activation': activation,
                    'Dropout': dropout_rate,
                    'Frozen Layers': frozen_layers,
                    'Test Accuracy': test_accuracy,
                    'Test Loss': test_loss
                }, ignore_index=True)

                # Saving the best model
                if test_accuracy > best_accuracy:
                    best_accuracy = test_accuracy
                    best_model_details = {
                        'Neurons': neurons,
                        'Activation': activation,
                        'Dropout': dropout_rate,
                        'Frozen Layers': frozen_layers,
                        'Test Set Accuracy': f"{test_accuracy * 100:.2f}%"
                    }
                    model.save(f'{base_dir}/Best_InceptionV3_Visual_Model.h5')

87910968/87910968 [==============================] - 0s 0us/step
Test accuracy: 92.00%, Test loss: 0.8233
Test accuracy: 95.20%, Test loss: 0.2030
Test accuracy: 91.20%, Test loss: 1.4841
Test accuracy: 92.00%, Test loss: 0.1771
Test accuracy: 92.00%, Test loss: 0.4073
Test accuracy: 89.60%, Test loss: 0.3049
Test accuracy: 88.80%, Test loss: 0.4312
Test accuracy: 88.80%, Test loss: 0.2720
Test accuracy: 86.40%, Test loss: 0.4835
Test accuracy: 92.00%, Test loss: 0.8425
Test accuracy: 90.40%, Test loss: 2.0492
Test accuracy: 91.20%, Test loss: 1.4574
Test accuracy: 93.60%, Test loss: 0.3910
Test accuracy: 92.00%, Test loss: 1.1069
Test accuracy: 89.60%, Test loss: 1.6758
Test accuracy: 92.80%, Test loss: 0.2129
Test accuracy: 91.20%, Test loss: 0.4318
Test accuracy: 89.60%, Test loss: 0.9262


In [8]:
# Displaying all results
print(results_df)

   Neurons Activation Dropout Frozen Layers  Test Accuracy  Test Loss
0      256       relu       0           311          0.920   0.823283
1      256       relu       0           309          0.952   0.203033
2      256       relu       0           306          0.912   1.484068
3      256       relu     0.3           311          0.920   0.177078
4      256       relu     0.3           309          0.920   0.407330
5      256       relu     0.3           306          0.896   0.304904
6      256       relu     0.5           311          0.888   0.431158
7      256       relu     0.5           309          0.888   0.272027
8      256       relu     0.5           306          0.864   0.483468
9      512       relu       0           311          0.920   0.842484
10     512       relu       0           309          0.904   2.049221
11     512       relu       0           306          0.912   1.457391
12     512       relu     0.3           311          0.936   0.391002
13     512       rel

In [9]:
# Displaying the best model's details
print("Best model details:", best_model_details)

Best model details: {'Neurons': 256, 'Activation': 'relu', 'Dropout': 0, 'Frozen Layers': 309, 'Test Set Accuracy': '95.20%'}


# **ResNet50 Model:**

In [4]:
# Function to preprocess the images to meet the requirements of the pertinent pre-trained model
def preprocess_image(file_path, label):
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize_with_pad(img, 224, 224, antialias=True) # Resizing to 224x224 for ResNet50
  img = preprocess_input_resnet(img)  # Applying the preprocess function for ResNet50
  return img, label

# Label Encoding the Class Variable
label_encoder = LabelEncoder()
labels = df['Class'].unique()
label_encoder.fit(labels)

# Function to prepare the train, validation and test datasets to be compatible with TensorFlow
def prepare_dataset(df, image_dir, preprocessing_function, batch_size=32, cache=True, shuffle=False):
    image_paths = df['Id'].apply(lambda x: f"{image_dir}/{x}").values
    labels = label_encoder.transform(df['Class'].values)
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
    dataset = dataset.map(preprocessing_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    if cache:
        dataset = dataset.cache()
    return dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [5]:
# Preparing the train, validation and test datasets, with shuffling for the train dataset
train_dataset = prepare_dataset(train_df, train_image_dir, preprocess_image, batch_size=batch_size, cache=True, shuffle=True)
val_dataset = prepare_dataset(val_df, val_image_dir, preprocess_image, batch_size=batch_size, cache=True)
test_dataset = prepare_dataset(test_df, test_image_dir, preprocess_image, batch_size=batch_size, cache=True)

In [6]:
# Clearing unnecessary memory usage before training the model
gc.collect()
tf.keras.backend.clear_session()

In [7]:
# Defining the parameter grid with various model combinations
neuron_options = [256, 512]
activation_options = ['relu']
dropout_options = [0, 0.3, 0.5]
# Loading the base ResNet50 model to determine the total number of layers
temp_model = ResNet50(weights='imagenet', include_top=False)
total_layers = len(temp_model.layers)
frozen_layers_options = [total_layers, total_layers - 2, total_layers - 5]  # Experimenting with the number of frozen layers for further fine-tuning

# Initializing a DataFrame to store the results of each model combination
results_df = pd.DataFrame(columns=['Neurons', 'Activation', 'Dropout', 'Frozen Layers', 'Test Accuracy', 'Test Loss'])

# Variables to keep track of the best model's accuracy and details
best_accuracy = 0
best_model_details = {}

for neurons in neuron_options:
    for activation in activation_options:
        for dropout_rate in dropout_options:
            for frozen_layers in frozen_layers_options:

                # Loading the base ResNet50 model
                base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

                # Freezing the layers for further fine-tuning
                for layer in base_model.layers[:frozen_layers]:
                    layer.trainable = False

                # Adding custom layers
                x = base_model.output
                x = Flatten()(x)
                x = Dense(neurons, activation=activation)(x)
                x = Dropout(dropout_rate)(x)
                predictions = Dense(len(label_encoder.classes_), activation='softmax')(x)

                # Complete model
                model = Model(inputs=base_model.input, outputs=predictions)
                model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

                # Fitting the model
                history = model.fit(
                    train_dataset,
                    validation_data=val_dataset,
                    epochs=25,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)],
                    verbose = 0
                )

                # Evaluating the model on the test set
                test_loss, test_accuracy = model.evaluate(test_dataset, verbose = 0)
                print(f"Test accuracy: {test_accuracy * 100:.2f}%, Test loss: {test_loss:.4f}")

                # Updating the results DataFrame
                results_df = results_df.append({
                    'Neurons': neurons,
                    'Activation': activation,
                    'Dropout': dropout_rate,
                    'Frozen Layers': frozen_layers,
                    'Test Accuracy': test_accuracy,
                    'Test Loss': test_loss
                }, ignore_index=True)

                # Saving the best model
                if test_accuracy > best_accuracy:
                    best_accuracy = test_accuracy
                    best_model_details = {
                        'Neurons': neurons,
                        'Activation': activation,
                        'Dropout': dropout_rate,
                        'Frozen Layers': frozen_layers,
                        'Test Set Accuracy': f"{test_accuracy * 100:.2f}%"
                    }
                    model.save(f'{base_dir}/Best_ResNet50_Visual_Model.h5')

Test accuracy: 96.00%, Test loss: 0.1683
Test accuracy: 96.00%, Test loss: 0.1682
Test accuracy: 94.40%, Test loss: 0.3475
Test accuracy: 97.60%, Test loss: 0.1171
Test accuracy: 96.00%, Test loss: 0.4765
Test accuracy: 93.60%, Test loss: 0.4985
Test accuracy: 96.00%, Test loss: 0.3298
Test accuracy: 92.00%, Test loss: 0.4952
Test accuracy: 95.20%, Test loss: 0.3764
Test accuracy: 94.40%, Test loss: 0.7463
Test accuracy: 96.00%, Test loss: 0.2710
Test accuracy: 94.40%, Test loss: 0.4725
Test accuracy: 93.60%, Test loss: 2.2460
Test accuracy: 95.20%, Test loss: 0.9702
Test accuracy: 96.00%, Test loss: 0.8577
Test accuracy: 91.20%, Test loss: 1.4642
Test accuracy: 96.00%, Test loss: 0.4452
Test accuracy: 93.60%, Test loss: 0.6054


In [8]:
# Displaying all results
print(results_df)

   Neurons Activation Dropout Frozen Layers  Test Accuracy  Test Loss
0      256       relu       0           175          0.960   0.168293
1      256       relu       0           173          0.960   0.168230
2      256       relu       0           170          0.944   0.347500
3      256       relu     0.3           175          0.976   0.117062
4      256       relu     0.3           173          0.960   0.476467
5      256       relu     0.3           170          0.936   0.498494
6      256       relu     0.5           175          0.960   0.329796
7      256       relu     0.5           173          0.920   0.495200
8      256       relu     0.5           170          0.952   0.376444
9      512       relu       0           175          0.944   0.746336
10     512       relu       0           173          0.960   0.271041
11     512       relu       0           170          0.944   0.472504
12     512       relu     0.3           175          0.936   2.245982
13     512       rel

In [9]:
# Displaying the best model's details
print("Best model details:", best_model_details)

Best model details: {'Neurons': 256, 'Activation': 'relu', 'Dropout': 0.3, 'Frozen Layers': 175, 'Test Set Accuracy': '97.60%'}


# **DenseNet121 Model:**

In [10]:
# Function to preprocess the images to meet the requirements of the pertinent pre-trained model
def preprocess_image(file_path, label):
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize_with_pad(img, 224, 224, antialias=True) # Resizing to 224x224 for DenseNet121
  img = preprocess_input_densenet(img)  # Applying the preprocess function for DenseNet121
  return img, label

# Label Encoding the Class Variable
label_encoder = LabelEncoder()
labels = df['Class'].unique()
label_encoder.fit(labels)

# Function to prepare the train, validation and test datasets to be compatible with TensorFlow
def prepare_dataset(df, image_dir, preprocessing_function, batch_size=32, cache=True, shuffle=False):
    image_paths = df['Id'].apply(lambda x: f"{image_dir}/{x}").values
    labels = label_encoder.transform(df['Class'].values)
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
    dataset = dataset.map(preprocessing_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    if cache:
        dataset = dataset.cache()
    return dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [11]:
# Preparing the train, validation and test datasets, with shuffling for the train dataset
train_dataset = prepare_dataset(train_df, train_image_dir, preprocess_image, batch_size=batch_size, cache=True, shuffle=True)
val_dataset = prepare_dataset(val_df, val_image_dir, preprocess_image, batch_size=batch_size, cache=True)
test_dataset = prepare_dataset(test_df, test_image_dir, preprocess_image, batch_size=batch_size, cache=True)

In [12]:
# Clearing unnecessary memory usage before training the model
gc.collect()
tf.keras.backend.clear_session()

In [13]:
# Defining the parameter grid with various model combinations
neuron_options = [256, 512]
activation_options = ['relu']
dropout_options = [0, 0.3, 0.5]
# Loading the base DenseNet121 model to determine the total number of layers
temp_model = DenseNet121(weights='imagenet', include_top=False)
total_layers = len(temp_model.layers)
frozen_layers_options = [total_layers, total_layers - 2, total_layers - 5]  # Experimenting with the number of frozen layers for further fine-tuning

# Initializing a DataFrame to store the results of each model combination
results_df = pd.DataFrame(columns=['Neurons', 'Activation', 'Dropout', 'Frozen Layers', 'Test Accuracy', 'Test Loss'])

# Variables to keep track of the best model's accuracy and details
best_accuracy = 0
best_model_details = {}

for neurons in neuron_options:
    for activation in activation_options:
        for dropout_rate in dropout_options:
            for frozen_layers in frozen_layers_options:

                # Loading the base DenseNet121 model
                base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

                # Freezing the layers for further fine-tuning
                for layer in base_model.layers[:frozen_layers]:
                    layer.trainable = False

                # Adding custom layers
                x = base_model.output
                x = Flatten()(x)
                x = Dense(neurons, activation=activation)(x)
                x = Dropout(dropout_rate)(x)
                predictions = Dense(len(label_encoder.classes_), activation='softmax')(x)

                # Complete model
                model = Model(inputs=base_model.input, outputs=predictions)
                model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

                # Fitting the model
                history = model.fit(
                    train_dataset,
                    validation_data=val_dataset,
                    epochs=25,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)],
                    verbose = 0
                )

                # Evaluating the model on the test set
                test_loss, test_accuracy = model.evaluate(test_dataset, verbose = 0)
                print(f"Test accuracy: {test_accuracy * 100:.2f}%, Test loss: {test_loss:.4f}")

                # Updating the results DataFrame
                results_df = results_df.append({
                    'Neurons': neurons,
                    'Activation': activation,
                    'Dropout': dropout_rate,
                    'Frozen Layers': frozen_layers,
                    'Test Accuracy': test_accuracy,
                    'Test Loss': test_loss
                }, ignore_index=True)

                # Saving the best model
                if test_accuracy > best_accuracy:
                    best_accuracy = test_accuracy
                    best_model_details = {
                        'Neurons': neurons,
                        'Activation': activation,
                        'Dropout': dropout_rate,
                        'Frozen Layers': frozen_layers,
                        'Test Set Accuracy': f"{test_accuracy * 100:.2f}%"
                    }
                    model.save(f'{base_dir}/Best_DenseNet121_Visual_Model.h5')

29084464/29084464 [==============================] - 0s 0us/step
Test accuracy: 96.80%, Test loss: 0.1207
Test accuracy: 94.40%, Test loss: 0.2201
Test accuracy: 95.20%, Test loss: 0.4653
Test accuracy: 94.40%, Test loss: 0.1393
Test accuracy: 95.20%, Test loss: 0.1335
Test accuracy: 93.60%, Test loss: 0.2537
Test accuracy: 92.00%, Test loss: 0.1645
Test accuracy: 90.40%, Test loss: 0.1723
Test accuracy: 93.60%, Test loss: 0.1620
Test accuracy: 95.20%, Test loss: 0.2859
Test accuracy: 96.80%, Test loss: 0.1456
Test accuracy: 91.20%, Test loss: 0.3687
Test accuracy: 96.00%, Test loss: 0.1015
Test accuracy: 96.80%, Test loss: 0.1684
Test accuracy: 92.00%, Test loss: 0.1748
Test accuracy: 91.20%, Test loss: 0.2575
Test accuracy: 91.20%, Test loss: 0.1886
Test accuracy: 92.80%, Test loss: 0.2084


In [14]:
# Displaying all results
print(results_df)

   Neurons Activation Dropout Frozen Layers  Test Accuracy  Test Loss
0      256       relu       0           427          0.968   0.120673
1      256       relu       0           425          0.944   0.220135
2      256       relu       0           422          0.952   0.465289
3      256       relu     0.3           427          0.944   0.139316
4      256       relu     0.3           425          0.952   0.133540
5      256       relu     0.3           422          0.936   0.253680
6      256       relu     0.5           427          0.920   0.164451
7      256       relu     0.5           425          0.904   0.172330
8      256       relu     0.5           422          0.936   0.162019
9      512       relu       0           427          0.952   0.285922
10     512       relu       0           425          0.968   0.145568
11     512       relu       0           422          0.912   0.368659
12     512       relu     0.3           427          0.960   0.101546
13     512       rel

In [15]:
# Displaying the best model's details
print("Best model details:", best_model_details)

Best model details: {'Neurons': 256, 'Activation': 'relu', 'Dropout': 0, 'Frozen Layers': 427, 'Test Set Accuracy': '96.80%'}


# **Model Ensembles:**

In [16]:
# Loading the label encodings from a previously made joblib file

label_encoder = joblib.load(f'{base_dir}/label_encoder.joblib')

# Loading the individual models

model_vgg19 = load_model(f'{base_dir}/Best_VGG19_Visual_Model.h5')

In [17]:
model_inceptionv3 = load_model(f'{base_dir}/Best_InceptionV3_Visual_Model.h5')

In [18]:
model_resnet50 = load_model(f'{base_dir}/Best_ResNet50_Visual_Model.h5')

In [19]:
model_densenet121 = load_model(f'{base_dir}/Best_DenseNet121_Visual_Model.h5')

In [20]:
# Functions to preprocess the images to meet the requirements of the pertinent pre-trained model
def preprocess_image_vgg19(file_path, label):
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize_with_pad(img, 224, 224, antialias=True)
  img = preprocess_input_vgg(img)
  return img, label

def preprocess_image_inceptionv3(file_path, label):
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize_with_pad(img, 299, 299, antialias=True)
  img = preprocess_input_inception(img)
  return img, label

def preprocess_image_resnet50(file_path, label):
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize_with_pad(img, 224, 224, antialias=True)
  img = preprocess_input_resnet(img)
  return img, label

def preprocess_image_densenet121(file_path, label):
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize_with_pad(img, 224, 224, antialias=True)
  img = preprocess_input_densenet(img)
  return img, label

# Function to prepare the test dataset to be compatible with TensorFlow
def prepare_dataset(df, image_dir, preprocessing_function, batch_size=32, cache=True, shuffle=False):
    image_paths = df['Id'].apply(lambda x: f"{image_dir}/{x}").values
    labels = label_encoder.transform(df['Class'].values)
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
    dataset = dataset.map(preprocessing_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    if cache:
        dataset = dataset.cache()
    return dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [21]:
# Preparing the test dataset to be compatible with each of the models using the above functions
test_dataset_vgg19 = prepare_dataset(test_df, test_image_dir, preprocess_image_vgg19, batch_size=batch_size, cache=True)
test_dataset_inceptionv3 = prepare_dataset(test_df, test_image_dir, preprocess_image_inceptionv3, batch_size=batch_size, cache=True)
test_dataset_resnet50 = prepare_dataset(test_df, test_image_dir, preprocess_image_resnet50, batch_size=batch_size, cache=True)
test_dataset_densenet121 = prepare_dataset(test_df, test_image_dir, preprocess_image_densenet121, batch_size=batch_size, cache=True)

In [22]:
# Making class probability predictions for each model
predictions_vgg19 = model_vgg19.predict(test_dataset_vgg19)
predictions_inceptionv3 = model_inceptionv3.predict(test_dataset_inceptionv3)
predictions_resnet50 = model_resnet50.predict(test_dataset_resnet50)
predictions_densenet121 = model_densenet121.predict(test_dataset_densenet121)

32/32 [==============================] - 4s 47ms/step


In [23]:
# Making class predictions for each model
class_predictions_vgg19 = np.argmax(model_vgg19.predict(test_dataset_vgg19), axis=1)
class_predictions_inceptionv3 = np.argmax(model_inceptionv3.predict(test_dataset_inceptionv3), axis=1)
class_predictions_resnet50 = np.argmax(model_resnet50.predict(test_dataset_resnet50), axis=1)
class_predictions_densenet121 = np.argmax(model_densenet121.predict(test_dataset_densenet121), axis=1)

32/32 [==============================] - 0s 13ms/step


In [24]:
# Using soft-voting to make predictions
ensemble_predictions = (predictions_vgg19 + predictions_inceptionv3 + predictions_resnet50 + predictions_densenet121) / 4

true_labels = test_df['Class'].values
true_labels_encoded = label_encoder.transform(true_labels)
predicted_classes = np.argmax(ensemble_predictions, axis=1)

ensemble_model_acc = accuracy_score(true_labels_encoded, predicted_classes)
print(f'Ensemble Model (Soft Voting) Accuracy: {ensemble_model_acc}')

Ensemble Model (Soft Voting) Accuracy: 0.976


In [25]:
# Using majority voting to make predictions
stacked_predictions = np.column_stack((class_predictions_vgg19, class_predictions_inceptionv3, class_predictions_resnet50, class_predictions_densenet121))
ensemble_predictions = mode(stacked_predictions, axis=1)[0].flatten()

true_labels = test_df['Class'].values
true_labels_encoded = label_encoder.transform(true_labels)

ensemble_model_acc = accuracy_score(true_labels_encoded, ensemble_predictions)
print(f'Ensemble Model (Majority Voting) Accuracy: {ensemble_model_acc}')

Ensemble Model (Majority Voting) Accuracy: 0.976


# **The ensemble model accuracy of 97.6% is tied with the individual ResNet50 model. However, it is still smaller than the highest hybrid model accuracy.**